In [1]:
# pip install pymorphy2 >> None

In [1]:
import pandas as pd
import os
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
# задаем директорию
file_path = 'C:/Users/Пользователь/Desktop/Hack_LinkedIn'

In [3]:
# открываем файлы с данными
#posts = pd.read_csv(f'{file_path}/posts.csv')
#profiles = pd.read_csv(f'{file_path}/profiles.csv')
posts = pd.read_csv('posts.csv')
profiles = pd.read_csv('profiles.csv')

In [4]:
# открываем датафрейм с постами
posts.head(2)

,Unnamed: 0,user_id,text,likes,comments,reposts
0,0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0
1,1,ali-wodan,I’m #hiring. Know anyone who might be interested?,1,0,0


In [5]:
# открываем датафрейм с информацией о пользователях
profiles.head(2)

,Unnamed: 0,id,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
0,0,ali-wodan,Ali Wodan,Head of Design,Performix,Head Of Design,"podcast, it","Москва, Московская область, Россия",2 391,500+,"I am a digital product design lead. I've been designing digital systems for years. Teams creating, management, design process building, user research, analytics etc. it's all that i like to do. Also I am into psychology, music, philosophy."
1,1,ikotow,Игорь Котов,Директор по производству – Технократия,Технократия,Технократия,"it, обучение, менеджмент, технологии, производство","Казань, Республика Татарстан, Россия",340,338,Making IT production great again! Создатель сервиса для управления ресурсами: Resourcer.appМой telegram-канал: https://t.me/pastikotow


In [6]:
# переименуем столбец text в post для лучшего отражения содержимого
posts = posts.rename(columns={'text': 'post'})

**Предобработка данных**

In [7]:
# функция удаления эмодзи
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # смайлики
                               u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                               u"\U0001F680-\U0001F6FF"  # транспорт и символы на карте
                               u"\U0001F1E0-\U0001F1FF"  # флаги
                               u"\U00002500-\U00002BEF"  # китайские символы
                               # другие разные символы
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642" 
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Удаляем эмодзи, используя паттерны
    text_without_emojis = emoji_pattern.sub(r'', text)
    return text_without_emojis

# удаляем эмодзи из постов
posts['post'] = posts['post'].apply(lambda x: remove_emojis(x) if pd.notnull(x) else x)

In [8]:
# функция лемматизации текста
morph = pymorphy2.MorphAnalyzer()
def lemmatize_text(text):
    lemmatized_words = [morph.parse(word)[0].normal_form for word in text.split()]
    return ' '.join(lemmatized_words)

# лемматизируем посты
posts['post_lemmatized'] = posts['post'].apply(lemmatize_text)

In [9]:
# удаляем лишние текстовые символы (те, которые не состоят из букв русского алфавита)
posts['post_lemmatized'] = posts['post_lemmatized'].str.replace('[^а-яА-ЯёЁ\s]', '', regex=True) # только русские буквы и пробелы

# Шаблон '\W' в методе str.replace() представляет любой символ, не состоящий из слов, в т.ч. пробелы.
# Шаблон '[^\w\s]' пробелы оставляет, '[^\w\s\d]' + цифры

In [10]:
# скачиваем стоп-слова 
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krasn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# еще один список от bukvarix.com - список стоп-слов Яндекс Wordstat - (этот список можно дополнить/изменить)
current_directory = os.getcwd()
#file_path_words = f'{file_path}\stop_words.txt'
file_path_words = f'stop_words.txt'
with open(file_path_words, 'r', encoding='utf-8') as file:
    stop_words_buk = file.read()

In [12]:
# удаляем стоп-слова и слова-паразиты
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_buk]))

In [13]:
# определяем шаблон регулярного выражения для русских букв
pattern = '[^а-яА-ЯёЁ]'

# создаем маску, чтобы проверить, содержит ли каждая ячейка русские буквы
mask = posts['post_lemmatized'].str.contains(pattern, regex=True)

# фильтруем датафрейм, используя маску
posts = posts[mask]

In [14]:
posts.sample(20)

,Unnamed: 0,user_id,post,likes,comments,reposts,post_lemmatized
1082,1082,enjirouz,"Наконец-то вышла первая порция долгожданной видео-лекции по стартапам.Первые 3 видео уже ждут вас на моем YouTube-канале:https://lnkd.in/eGW_kZ_GВ них я помогаю вам найти свою бизнес-идею, определить спрос на продукт и проанализировать конкурентов.В течение недели выйдут и остальные 14 видео.Лекции можно слушать как подкаст.А попрактиковаться можно с помощью рабочей тетради, которая ждет вас на Boosty: https://lnkd.in/eYygVvgN",0,0,0,наконецто выйти первый порция долгожданный видеолекция стартапампервые видео ждать каналеть помогать найти бизнесидею определить спрос продукт проанализировать конкурентовть течение неделя выйти остальной видеолекция слушать подкаст попрактиковаться помощь рабочий тетради ждать
791,791,melekes,Написал про работу с HBase в Erlang'е используя Thrift #erlang #thrift #hbase,0,0,0,написать работа использовать
1275,1275,vladimirmoiseev,Работает как часы.),2,0,0,работать часы
4074,4074,marina-a-577a64232,"""self-help”, или «no smoking»:то, что не делает нас сильней#it #work #psychology #addiction #network #help #helpme #alcohol #develop #smoke #brain #vino #blissА Вы знали, что все-все наши зависимости работают одинаково, и за них отвечает одни и те же зоны головного мозга? И мозгу неважно какая она: алкогольная, никотиновая, пищевая, наркотическая или любая другая из существующих, даже ХОРОШАЯ зависимость, «а-ля» тренировки или обучение каждый день – есть и такие (есть ребята, у которых по 20 тренировок или 100 часов обучения в неделю), это все равно зависимость. Мозгу все равно, чем вы будете «кайфовать», ему главное получить это.Как работают зависимости? Давайте разберемВ норме доступные всем удовольствия жизни способствуют выработке клетками головного мозга специальных веществ – «гормонов радости» –эндорфинов, которые в свою очередь возбуждают нервные клетки центра удовольствия. Информация из центра удовольствия передаётся в другие участки мозга. В итоге мозг фиксирует соответствующую эмоциональную реакцию. И если эта реакция происходит ПОСТОЯННО, то возникает рефлекс: курю (пью,ем и т.д.) == ""кайфую"". Прям как в эксперименте про собаку Павлова.Когда вдруг человек прерывает стереотип привычной стимуляции центра удовольствия, его организм испытывает так называемый синдром отмены – психический дискомфорт (депрессию, тревогу, раздражительность), а в частых случаях и физический («ломка»). Вчера начала читать исследования в области наркологии. Готовы к одной интересной вещи? Если Вы употребляете алкоголь в ЛЮБОМ количестве чаще, чем раз в 10-14 дней, то вы вправе называться АЛКОГОЛИКОМ. ТОчКА.Если говорить о зависимостях, то, думаю, курение «забирает» больше всего сил и здоровья из всех существующий зависимостей (про «солевых» и героиновых наркоманов я не говорю), проверено на себе. Причина в том, что мы это делаем ежечасно. Думаю, выпить бутылку хорошего вина с любимым человеком раз в неделю (и быть алкоголиком))) все же лучше. Вспоминаю это чувство постоянной усталости, когда ты куришь.Выводы делайте сами. Любая зависимость – это зависимость, но лучше из двух зол выбрать наименьшее. Вопрос о дозах,регулярности и последствиях.P.S. Вот такие разные картинки зависимости выдала нейросеть. Прям как в книге (фильме) «Портрет Дориана Грея».",1,0,0,делать сильней знали всёвсё зависимость работать одинаково отвечать зона головной мозга мозг неважно алкогольная никотиновая пищевая наркотический любой существующих хороший зависимость аля тренировка обучение каждый день ребята тренировка час обучение неделю всё равно зависимость мозг всё равно кайфовать главное получить этокак работать зависимости давать разберемть норма доступный удовольствие жизнь способствовать выработка клетка головной мозг специальный вещество гормон радости эндорфинов очередь возбуждать нервный клетка центр удовольствия информация центр удовольствие передаваться участок мозга итог мозг фиксировать соответствовать эмоциональный реакцию реакция происходить пос

In [15]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2073 entries, 0 to 6176
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       2073 non-null   int64 
 1   user_id          2073 non-null   object
 2   post             2073 non-null   object
 3   likes            2073 non-null   object
 4   comments         2073 non-null   int64 
 5   reposts          2073 non-null   int64 
 6   post_lemmatized  2073 non-null   object
dtypes: int64(3), object(4)
memory usage: 129.6+ KB


Объединяем датафреймы

In [16]:
# переименуем столбец id в user_id в датафрейме profiles, для последующего объединения с posts
profiles = profiles.rename(columns={'id': 'user_id'})

In [17]:
# объединяем датафреймы
df = pd.merge(posts, profiles, on='user_id')

In [18]:
# удаляем лишние столбцы
df = df.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)

In [19]:
# удаляем дубликаты
df.drop_duplicates(inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2062 entries, 0 to 2072
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           2062 non-null   object
 1   post              2062 non-null   object
 2   likes             2062 non-null   object
 3   comments          2062 non-null   int64 
 4   reposts           2062 non-null   int64 
 5   post_lemmatized   2062 non-null   object
 6   user_name         2062 non-null   object
 7   user_head         2062 non-null   object
 8   user_work         1981 non-null   object
 9   user_position     2062 non-null   object
 10  user_tags         477 non-null    object
 11  user_location     2055 non-null   object
 12  user_viewers      2062 non-null   object
 13  user_contacts     2062 non-null   object
 14  user_common_info  1873 non-null   object
dtypes: int64(2), object(13)
memory usage: 257.8+ KB


In [21]:
df.sample(5)

,user_id,post,likes,comments,reposts,post_lemmatized,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
1801,yugoru,"А я опять с приглашением на очень классный курс! #womenintech #womenintechrus Зона комфорта на работе – мечта или реальность? Программа для женщин в мужском корпоративном окружении.Сколько раз за время работы вы слышали “Выйди из зоны комфорта”? А были ли вы когда-нибудь в “зоне комфорта”, чтобы из нее выйти?Если вы чувствуете, что в мужском корпоративном окружении вы не можете быть собой и не достигаете в карьере и жизни всего, чего могли и хотели бы; если испытываете постоянное чувство вины, что не уделяете достаточно времени ни семье, ни работе, то эта программа для вас.Цель программы:Понять и реализовать свою зону комфорта в работе.Во время работы в программе вы:- разработаете собственную модель жизненного баланса, учитывающую интересы и семьи, и карьеры;- преодолеете внутренние ограничения и обретете уверенность в собственном профессионализме;- выявите возможности для личностного и профессионального роста, основанного на собственной уникальности и приоритетах.В результате вы получите:- гармоничное сочетание различных сфер жизни;- повышение собственной ценности в команде и укрепление своей самооценки;- ощутимый прогресс в профессиональном развитии с опорой на вашу уникальность;Формат:- 5 онлайн вебинаров и 2 групповых менторинга;- Индивидуальные консультации;- Задания для самостоятельной работы.Даты:14 марта - 25 апреля 2022 (6 недель)Заявки принимаются с 1 по 22 февраля 2022.Программа бесплатная для членов сообществ Women in Big Data и Women in Tech. Участницы будут выбраны на условиях конкурсного отбора.Автор программы:Марина Алексеева, коуч и ментор профессионального благополучия, член оргкомитета сообщества Women in Big Data, экс-вице-президент Intel.Больше информации и регистрация -",5,0,1,приглашение очень классный курс зона комфорт работа мечта реальность программа женщина мужской корпоративный окружениисколько время работа слышать выйти зона комфорта когданибудь зона комфорта выйтиести чувствуете мужской корпоративный окружение достигать карьера жизнь хотеть испытывать постоянный чувство вины уделять достаточно время семье работе программа васцель программыпонять реализовать зона комфорт работевый время работа программа разработать собственный модель жизненный баланса учитывать интерес семьи карьеры преодолеть внутренний ограничение обрести уверенность собственный профессионализме выявить возможность личностный профессиональный роста основать собственный уникальность приоритетахть результат получите гармоничный сочетание различный сфера жизни повышение собственный ценность команда укрепление самооценки ощутимый прогресс профессиональный развитие опора ваш уникальностьформат онлайн вебинар групповой менторинга индивидуальный консультации задание самостоятельный работыдаты март апрель недельзаявка приниматься февраль программа бесплатный член сообщество участница выбрать условие конкурсный отбораавтор программымарин алексеева коучий ментор профессиональный благополучия член оргкомитет сообщество эксвицепрезидент больша информация регистрация,Юлия Горшкова,Founder GradeBuilder | Lead QA – AstralSoft | PM – Women in Tech Russia,АСТРАЛ-СОФТ,Senior Quality Assurance Engineer,"it, qa, education, womenintech, volunteering","Сан-Хавьер, Мурсия, Испания",1 649,500+,https://gradebuilder.tech/http://qahacking.ru/
1236,mixailtimofeev,"Скорочення витрат у логістиці за допомогою LEAN.LEAN - це система управління, яка покликана забезпечити максимальну ефективність діяльності компанії, зменшення втрат та оптимізацію всіх процесів. Застосування LEAN в логістиці може допомогти підприємствам скоротити витрати та збільшити прибуток.Одним з ключових принципів LEAN є відмова від непотрібних операцій. У логістиці це може означати виключення непотрібних етапів доставки, таких як надмірне перепакування чи зайві перевезення. Крім того, застосування LEAN може знизити 

In [22]:
# сохраняем датасет
df.to_csv('linkedin.csv')